<a href="https://colab.research.google.com/github/llillillj/open_campus_2023/blob/main/quiz_game_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# アナロジークイズゲーム

# 環境構築

In [ ]:
#@title
# 外部ライブラリのインストール
!pip install mecab-python3
!pip install fugashi[unidic-lite]

# 利用するライブラリのインポート
import gzip
import shutil
import os
import pandas as pd
from collections import Counter
import random

import ipywidgets
from IPython.display import HTML

from gensim.models import KeyedVectors
import sqlite3
import numpy as np

# 必要なファイル等の読み込みをする初期関数
# ドライブに保存されます
def initial_setup():
    necessary_files = [
        "wnjpn.db",
        "jawiki.all_vectors.100d.txt.bz2",
        "corpus/ldcc-20140209.tar.gz",
    ]
    if all(list(map(lambda path: os.path.exists(os.path.join("/content", path)),
                   necessary_files ))):pass
    else:
        # WordNet
        !wget -P /content https://raw.githubusercontent.com/llillillj/open_campus_2023/main/content/words_for_game.csv
        !wget -P /content https://github.com/singletongue/WikiEntVec/releases/download/20190520/jawiki.all_vectors.100d.txt.bz2
        # word2vecのダウンロード
        !wget -P /content https://github.com/bond-lab/wnja/releases/download/v1.1/wnjpn.db.gz
        with gzip.open(os.path.join("/content", 'wnjpn.db.gz'), 'rb') as f_in:
            with open(os.path.join("/content", 'wnjpn.db'), 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

# 必要なファイルの存在確認と読み込み
initial_setup()
# word2vecの実装
wv = KeyedVectors.load_word2vec_format(os.path.join("/content", 'jawiki.all_vectors.100d.txt.bz2'), binary=False)

# 出現する単語の取得
all_words = []
with open("/content/words_for_game.csv", "r") as f:
    all_words = f.read()

all_words = all_words.split("\n")[1:]

--2023-07-15 07:31:57--  https://raw.githubusercontent.com/llillillj/open_campus_2023/main/content/words_for_game.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 377674 (369K) [text/plain]
Saving to: ‘/content/words_for_game.csv.2’

words_for_game.csv. 100%[===================>] 368.82K  --.-KB/s    in 0.03s   

2023-07-15 07:31:58 (10.5 MB/s) - ‘/content/words_for_game.csv.2’ saved [377674/377674]

--2023-07-15 07:31:58--  https://github.com/singletongue/WikiEntVec/releases/download/20190520/jawiki.all_vectors.100d.txt.bz2
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-productio

# 利用する関数の定義
|関数名|役割|
|-|-|
|start_game|**これを実行したらゲーム開始**|
|init_game|ゲーム環境の初期化|
|base_html|HTMLのベースを作成|
|get_formula_and_ans|問題式，正解例，文章の作成|
|remove_hash|エンティティから#を除去|
|get_formula|問題式の取得|
|get_options|選択肢の取得|
|get_hype_list|上位語を取得|
|get_hypo_list|下位語を取得|

In [ ]:
#@title
# 内部的なシステム
def get_hype_list(target_word: str):
    """上位語を取得する関数"""
    hype_list = []

    conn = sqlite3.connect(os.path.join("/content", 'wnjpn.db'))
    cursor = conn.cursor()

    query = f"""
    SELECT
      w1.lemma, sl.link, w2.lemma
    FROM synlink AS sl
    INNER JOIN synset AS sy1 ON sy1.synset = sl.synset1
    INNER JOIN synset AS sy2 ON sy2.synset = sl.synset2
    INNER JOIN sense AS se1 ON se1.synset = sy1.synset
    INNER JOIN sense AS se2 ON se2.synset = sy2.synset
    INNER JOIN word AS w1 ON w1.wordid = se1.wordid
    INNER JOIN word AS w2 ON w2.wordid = se2.wordid
    WHERE w1.lemma = '{target_word}' AND sl.link IN ('hype', 'hypo')
      AND se1.lang = 'jpn' AND w1.lang = 'jpn' AND se2.lang = 'jpn' AND w2.lang = 'jpn'
    """

    cursor.execute(query)
    results = cursor.fetchall()

    conn.close()

    for row in results:
        _, link, lemma2 = row
        if link == 'hype': hype_list.append(lemma2)
    return hype_list


def get_hypo_list(hype: str):
    """ 下位語を取得して，そのリストを返却"""
    hypo_list = []

    conn = sqlite3.connect(os.path.join("/content", 'wnjpn.db'))
    cursor = conn.cursor()

    query = f"""
    SELECT
      w1.lemma, sl.link, w2.lemma
    FROM synlink AS sl
    INNER JOIN synset AS sy1 ON sy1.synset = sl.synset1
    INNER JOIN synset AS sy2 ON sy2.synset = sl.synset2
    INNER JOIN sense AS se1 ON se1.synset = sy1.synset
    INNER JOIN sense AS se2 ON se2.synset = sy2.synset
    INNER JOIN word AS w1 ON w1.wordid = se1.wordid
    INNER JOIN word AS w2 ON w2.wordid = se2.wordid
    WHERE w1.lemma = '{hype}' AND sl.link IN ('hype', 'hypo')
      AND se1.lang = 'jpn' AND w1.lang = 'jpn' AND se2.lang = 'jpn' AND w2.lang = 'jpn'
    """

    cursor.execute(query)
    results = cursor.fetchall()

    conn.close()

    for row in results:
        _, link, lemma2 = row
        if link == 'hypo': hypo_list.append(lemma2)
    return hypo_list

def get_options(w1, w2, w3, ans):
    """選択肢の作成"""
    options = []
    for word in [ans, w1, w2, w3]:
        if word == ans: print(ans)
        for hype in get_hype_list(word):
            if word == ans: options.extend(get_hypo_list(hype)[:1])
            else: options.extend(get_hypo_list(hype)[:2])
            options = list(set(options))
            if len(options) >= 8:
                options = options[:8]
                if ans in options:
                    break
                else:
                    options = options[:7]
                    options += [ans]
        if len(options) >= 8:break
    random.shuffle(options)
    return options

def get_formula(w1, w2, ans):
    """"""
    d = dict(w1=w1, w2=w2)
    keys = list(d.keys())
    random.shuffle(keys)
    s = np.random.randint(1, 3)
    if s==1:
        params = dict(positive=list(d.values()), negative=[ans])
        simw = wv.most_similar(topn=1, **params)[0][0].replace("#", "")
        pos_list = list(d.values())
        neg = simw
    else:
        parmas = dict(positive=[d[keys[0]], ans], negative=[d[keys[1]]])
        simw = wv.most_similar(topn=1, **parmas)[0][0].replace("#", "")
        pos_list = [d[keys[1]], simw]
        neg = d[keys[0]]
    random.shuffle(pos_list)
    formula = f"{pos_list[0]} - {neg} + {pos_list[1]} = ?"
    words = [pos_list[0], neg, pos_list[1]]
    text = f"{neg}にとって{pos_list[0]}は，{pos_list[1]}にとってのなに？"
    words = [pos_list[0], neg, pos_list[1], ans]
    return text, formula, words

def get_formula_and_ans(all_words=all_words):
    """問題文の作成"""
    i1, i2, i3 = np.random.choice(np.arange(len(all_words)), 3, replace=False)
    w1 = all_words[i1]
    w2 = all_words[i2]
    ans = all_words[i3]
    text, formula, words = get_formula(w1, w2, ans)
    return ans, text, formula, words

def remove_hash(*args):
    """「#」付きの文字に対して「#」を削除"""
    return list(map(lambda w: w.replace("#", ""), args))

def init_game():
    """ゲームの初期化関数"""
    ans, text, formula, words = get_formula_and_ans()
    options = get_options(words[0], words[1], words[2], ans)
    return ans, options, formula, text

# ゲームの表示用
def base_html(inner, add_style={}):
    """
    divタグの作成
        inner(str): divタグ内に来るもの
        add_style(dict): ベースとなるstyleに追加
    Return:
    """
    # styleの作成
    style = "background-color: #3af; color: #333; padding:10px; border-radius: 8px; text-align: center; font-weight: bold;"
    for key, value in add_style.items():
        incoming_style = " {}: {};".format(key, value)
        style += incoming_style

    # htmlの作成(divタグとその中身)
    html = f'''<div style="{style}"> {inner} </div>'''
    return html

# 初期セットアップまとめ
def start_game():
    ans, options, formula, sub_formula = init_game()

    def show_html(your_ans=None):
        header_html = base_html("アナロジーゲーム", add_style={"font-size": "30px"})

        option_html_generator = lambda text: base_html(f"{text}", {"border": "solid white", "color": "white", "margin": "1vw", "width": "18vw", "font-size": "20px"})
        options_html = "".join([option_html_generator(option_text) for option_text in options])
        options_area = base_html(options_html, {"display": "flex", "flex-wrap": "wrap", "padding": "0px 2.5vw"})

        formula_html = base_html(f"<u>{formula}</u>", {"font-size": "45px"})
        sub_formula_html = base_html(sub_formula, {"font-size": "20px"})
        main_html = base_html("".join([formula_html, sub_formula_html]))

        correct_html = base_html("正解", {"background-color": "#9f9", "font-size": "70px", "width": "300px", "position": "absolute", "top": "135px", "left": "50vw", "margin-left": "-150px", "box-shadow": "2px 2px 2px 2px"})
        failed_html = base_html(f"不正解<span style='font-size: 15px;'>正解は{ans}</span>", {"background-color": "#f99", "font-size": "70px", "width": "300px", "position": "absolute", "top": "135px", "left": "50vw", "margin-left": "-150px", "box-shadow": "2px 2px 2px 2px"})

        html_components = [header_html, main_html, options_area]

        if your_ans is None:
            pass
        elif your_ans == ans:
            html_components.append(correct_html)
        else:
            html_components.append(failed_html)

        display(HTML(base_html(base_html("".join(html_components)), {"padding": "2vw", "background-color": "blue"})))

    ipywidgets.interact(show_html, your_ans=options)

# アプリの実行

In [ ]:
start_game()

心情


interactive(children=(Dropdown(description='your_ans', options=('物質主義', '情感', '影', '純潔', '傾向', 'プラス', '便宜', '心…